In [1]:
MAIN_PATH = "/home/carlos/MasterDS/tfm"
JSON_DATA_PATH = '{}/data/json/'.format(MAIN_PATH)
CSV_DATA_PATH = '{}/data/csv/'.format(MAIN_PATH)

In [2]:
import sys
sys.path.insert(0, MAIN_PATH)

In [3]:
%load_ext autoreload
%autoreload 2
from scripts.models.ltr.train import LTRTrain

%reload_ext autoreload

In [4]:
import pickle

# Train

In [5]:
ltr_params = {
    'key_events': ['goal', 'red_card', 'penalty'],
    'lags': [1, 3, 5],
    'target_metric': 'rouge',
    'drop_teams': True,
    'lemma': True,
    'metric_params': {'rouge_mode': 'rouge-1', 'rouge_metric': 'r'},
    'count_vec_kwargs': {'ngram_range': (1, 2), 'strip_accents': 'unicode'}
}
cat_features_dict = {'is_key_event': [0, 1],
                     'n_players_cat': ['no_player', 'one_player', 'more_than_one_player']}
num_features = ['tfidf_sum']
model_params = {
    'n_estimators': 100,
    'max_depth': 2,
    'bootstrap': True,
    
}

In [ ]:
# Uncomment if using cv
"""
model_params = {
    'n_estimators': [100, 200],
    'max_depth': [2, 5],
    'bootstrap': [True],
    
}
cv = 5
opt_metric = 'neg_mean_squared_error'
ltr_train = LTRTrain(cat_features=cat_features, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params,
                    cv=cv, opt_metric=opt_metric)
"""

In [6]:
ltr_train = LTRTrain(cat_features_dict=cat_features_dict, num_features=num_features,
                     model_params=model_params, ltr_params=ltr_params)

Setting target metric to rouge


In [166]:
%%time
ltr_train.train()

Reading /home/carlos/MasterDS/tfm/data/csv/summaries/ltr/features_targets/341d2aa93d/train.csv
Categorizing n_players...
Computing new length...
Dropping {'length', 'position', 'total_length', 'players_importance', 'sim_previous_3', 'json_file', 'score', 'n_stop', 'equalize', 'sim_previous_1', 'url', 'sim_previous_5', 'n_players', 'sentence_ix', 'advantage', 'event_ix'}
Training model...
Writing model to /home/carlos/MasterDS/tfm/models/ltr_random_forest/8a68cad9f5/ckpt.pickle
Writing model info to /home/carlos/MasterDS/tfm/models/ltr_random_forest/8a68cad9f5/model_info.pickle
CPU times: user 9.15 s, sys: 15.2 ms, total: 9.17 s
Wall time: 9.16 s


In [159]:
model = pickle.load(open(ltr_train.model_path, 'rb'))

In [160]:
model

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder(categories=[[0,
                                                                                              1],
                                                                                             ['no_player',
                                                                                              'one_player',
                                                                                              'more_than_one_player']],
                                                                                 drop='first'))]),
                                                  ['is_key_event',
                                                   'n_players_cat'])])),
  

In [132]:
oh_cols = model['preprocessing']

In [145]:
model['preprocessing'].named_transformers_['cat']['encoder'].get_feature_names()

array(['x0_1', 'x1_one_player', 'x1_more_than_one_player'], dtype=object)

In [144]:
model['preprocessing'].named_transformers_['cat']['encoder'].categories_

[array([0, 1]),
 array(['no_player', 'one_player', 'more_than_one_player'], dtype=object)]

In [148]:
cat_pipeline = Pipeline(steps=[
            ('encoder', OneHotEncoder(drop='first',
                                      categories=[[0, 1], ['no_player', 'one_player', 'more_than_one_player']]))
        ])
preprocessor = ColumnTransformer(transformers=[('cat', cat_pipeline, ltr_train.cat_features)],
                                         remainder='passthrough')

In [149]:
preprocessor.fit_transform(X_train)

array([[0.        , 1.        , 0.        , 3.5126526 ],
       [0.        , 1.        , 0.        , 2.19859856],
       [0.        , 0.        , 1.        , 5.068378  ],
       ...,
       [0.        , 1.        , 0.        , 3.65516568],
       [0.        , 1.        , 0.        , 2.20688526],
       [0.        , 1.        , 0.        , 2.19669828]])

In [150]:
X_train

,is_key_event,n_players_cat,tfidf_sum
0,0,one_player,3.512653
1,0,one_player,2.198599
2,0,more_than_one_player,5.068378
3,0,one_player,2.558036
4,0,one_player,2.188177
...,...,...,...
296655,0,one_player,3.147186
296656,0,one_player,3.549072
296657,0,one_player,3.655166
296658,0,one_player,2.206885


In [146]:
model['model'].feature_importances_

array([0.25574172, 0.        , 0.        , 0.74425828])

In [130]:
ltr_train.features

['is_key_event', 'n_players_cat', 'tfidf_sum']

In [7]:
model_info = pickle.load(open(ltr_train.model_info_path, 'rb'))

In [8]:
# La salida de mse de una etapa de cv en sklearn es negativa
model_info

{'best_score': None,
 'best_params': None,
 'model_out':                          Gini-importance
 x0_1                            0.255742
 x1_one_player                   0.000000
 x1_more_than_one_player         0.000000
 tfidf_sum                       0.744258}

In [9]:
model_info['model_out']

,Gini-importance
x0_1,0.255742
x1_one_player,0.000000
x1_more_than_one_player,0.000000
tfidf_sum,0.744258
